In [ ]:
# TODO: Revisar que el nuevo new_ns que eliga (antiguo 2800) 
# No sea tan grande, revisar en los df cuales son el depth maximo con el New_ns y
# Cuanto necesita el zlims[-1]
# Hacer un recorrido y guardar el new_ns mas chico que cubra todos los zlims.

In [1]:
%load_ext autoreload
%autoreload 2

from deep_bf.data_handler import DataLoader
import torch
import numpy as np
from tqdm import tqdm
import h5py
import hdf5plugin

data_path = "/home/panda/rf_data/dataset/raw"
img_path = "/home/panda/rf_data/dataset/img"

dl = DataLoader("/home/panda/rf_data/")

nz = 2048
nx = 256
new_Ns = 2800

device = "cuda"
dtype = torch.float32

In [2]:
df = dl.get_df()
df = df.query("RF == 1 and nc == 128 and source == 'CUBDL'")
df = df[df["name"].str[:3] != "JHU"]
df = df[df["name"] != "OSL010"]
df = df[df["name"].str[:3] != "UFL"]

names = df["name"]
print(len(names))

538


In [3]:
from deep_bf.wrapper import BenchmarkWrapper
from tqdm import tqdm

beamformer = "MVB"
bfs = [beamformer]
max_ns = 0

data = {}
for name in tqdm(names):
    #print(name)
    pw = dl.get_defined_pwdata(name, "RF")
    #print(pw.data.shape)

    f_num = 1.75
    window = "tukey25"

    angles_idx = np.arange(pw.n_angles)
    bw = BenchmarkWrapper(nz, nx, pw, angles_idx, bfs, f_num, window, device=device, dtype=dtype)
    
    angle = pw.n_angles // 2
    samples_idx = bw.get_samples_idx_by_angle(angle)
    #print(samples_idx.shape)

    #print(pw.zlims)

    #print(samples_idx.max())
    _ns = samples_idx.max().item()
    if max_ns <= _ns:
        max_ns = _ns

    data[name] = _ns

100%|██████████| 538/538 [00:18<00:00, 28.71it/s]


In [4]:
print(max_ns)
print(data)

2294.102294921875
{'MYO001': 2294.102294921875, 'MYO002': 2294.102294921875, 'MYO003': 2294.102294921875, 'MYO004': 2294.102294921875, 'MYO005': 2294.102294921875, 'INS004': 1535.0, 'INS006': 1535.0, 'INS008': 1535.0, 'INS015': 1919.0, 'INS016': 1919.0, 'INS019': 1919.0, 'INS021': 1919.0, 'OSL007': 1919.0, 'TSH002': 1547.807373046875, 'MYO006': 2294.102294921875, 'INS001': 1535.0, 'INS002': 1535.0, 'INS003': 1535.0, 'INS005': 1535.0, 'INS007': 1535.0, 'INS009': 1535.0, 'INS010': 1535.0, 'INS011': 1535.0, 'INS012': 1535.0, 'INS013': 1535.0, 'INS014': 1919.0, 'INS017': 1919.0, 'INS018': 1919.0, 'INS020': 1919.0, 'INS022': 1919.0, 'INS023': 1919.0, 'INS024': 1919.0, 'INS025': 1919.0, 'INS026': 1919.0, 'OSL002': 1919.0, 'OSL003': 1919.0, 'OSL004': 1919.0, 'OSL005': 1919.0, 'OSL006': 1919.0, 'TSH003': 1547.807373046875, 'TSH004': 1547.807373046875, 'TSH005': 1547.807373046875, 'TSH006': 1547.807373046875, 'TSH007': 1547.807373046875, 'TSH008': 1547.807373046875, 'TSH009': 1547.807373046875,